<a href="https://colab.research.google.com/github/technisekai/sentiment-analysis-of-new-halal-logo/blob/main/3_pengolahan_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3. Pengolahan Data**

Proses Sebelumnya : [**2. Pengolahan Data**](https://colab.research.google.com/drive/14v2qJpYfPWSzGsG04yGmc2OwPfomRnbW)

## 3.1 Install dan Impor Library

In [ ]:
# untuk menggunakan model indonesian sentiment
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 29.3 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# download stopword
!gdown --id 19nlMainHfItPZ4_9K1yS3WZgk8nJW1p5
# download normalisasi kata
!gdown --id 1cINKFxiSycaOwVY5O5SPAxEwuhKRvqSz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=19nlMainHfItPZ4_9K1yS3WZgk8nJW1p5
To: /content/stopwordbahasa.csv
100% 6.45k/6.45k [00:00<00:00, 9.34MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cINKFxiSycaOwVY5O5SPAxEwuhKRvqSz
To: /content/new_kamusalay.csv
100% 286k/286k [00:00<00:00, 71.3MB/s]


In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.3 MB/s 


In [ ]:
import cv2
import re
import pandas as pd
import imblearn
import warnings
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import metrics as m
from wordcloud import WordCloud
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from google.colab.patches import cv2_imshow
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from collections import Counter

warnings.filterwarnings('ignore')

# preparing var to stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# preparing normalize word
alay_dict = pd.read_csv('new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
# preparing remove stopword
id_stopword_dict = pd.read_csv('stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

## 3.2 Data Cleansing

meliputi proses mengubah huruf 

*   mengubah huruf kapital menjadi huruf kecil.
*   menghilangkan kata tidak penting seperti \n, simbol retweet, username, url, dan spasi ekstra.
*   Normalisasi kata
*   Stemming

In [ ]:
# impor dataset
df = pd.read_csv('/content/drive/MyDrive/Dataset/tugas-akhir/labeled/logo-halal-labeled-with-indonesia-sentiment.csv')

In [ ]:
import re

# lowercase
def lowercase(text):
    return text.lower()

# remove uncertain word in sentence
def remove_unnecessary_char(text):
    text = re.sub('\n',' ',text) # Remove every '\n'
    #text = re.sub('rt',' ',text) # Remove every retweet symbol
    text = re.sub('@[^\s]+',' ',text) # Remove every username
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) # Remove every URL
    text = re.sub('  +', ' ', text) # Remove extra spaces
    return text

# remove non-alphanumeruic   
def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    return text

# stemming
def stemming(text):
    return stemmer.stem(text)

# normalization
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

# remove stopword
def remove_stopword(text):
    text = ' '.join(['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text) # Remove extra spaces
    text = text.strip()
    return text

# preprocessing using function defined above
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_unnecessary_char(text) # 2
    text = remove_nonaplhanumeric(text) # 3
    text = normalize_alay(text) # 4
    text = remove_stopword(text) # 6
    text = stemming(text) # 5
    return text

In [ ]:
# memanggil fungsi prepoecess untuk cleansing kolom normalization_tweet
df['clean_tweet'] = df['normalization_tweet'].apply(preprocess)

In [ ]:
# menampilkan data
df[['normalization_tweet', 'clean_tweet']].head()

,normalization_tweet,clean_tweet
0,eh… saya bukan muslim tapi saya hanya ingin me...,eh muslim dapat suka logo halal
1,"intinya bukan labelnya yang budaya bang, tetap...",inti label budaya bang bentuk logo sih selera ...
2,"ayo jangan terkecoh sama isu adzan, logo halal...",ayo kecoh isu azan logo halal kemah titik 0 be...
3,"dengan terbitnya logo baru, menteri agama yaqu...",terbit logo menteri agama yaqut cholil qoumas ...
4,logo halal islam nusantara...alias liberal...,logo halal islam nusantara alias liberal


## 3.3 Data Labeling

In [ ]:
!git lfs install
!git clone https://huggingface.co/sahri/indonesiasentiment
!pip install transformers

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'indonesiasentiment'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 48 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (48/48), done.
Filtering content: 100% (2/2), 951.34 MiB | 29.24 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline

pretrained_name = "/content/indonesiasentiment"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)


In [ ]:
# definisi fungsi untuk labeling
def labeling(text):
  x = nlp(text)
  return x[0]['label']

In [ ]:
# panggil fungsi labeling dan buat kolom baru
df['sentiment'] = df['normalization_tweet'].apply(labeling)

## 3.4 Split Data 80:20

Memisah data latih dan uji dengan proporsi 80:20.

In [ ]:
# ubah label menjadi numeric
def label_to_num(label):
  if label == 'positive':
    return 1
  elif label == 'negative':
    return -1
  else:
    return 0

In [ ]:
# memanggil fungsi label_to_num
df['label'] = df['sentiment'].apply(label_to_num)

In [ ]:
# definisi atribut dan label
X = df['clean_tweet'].values
y = df['label'].values

In [ ]:
# split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# tampilkan jumlah data
print("Jumlah data latih : ", len(X_train))
print("Jumlah data Uji : ", len(X_test))

Jumlah data latih :  800
Jumlah data Uji :  200


## 3.5 TF-IDF

Mengubah kalimat menjadi vector angka numerik.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# ubah teks ke vektor berdasar data train
tf_idf = TfidfVectorizer()
tf_idf.fit(X_train)

TfidfVectorizer()

In [ ]:
# ubah teks ke vektor
X_train = tf_idf.transform(X_train).toarray()
X_test = tf_idf.transform(X_test).toarray()

## 3.6 Over and Under Sampling

Menyamakan proporsi data antara sentimen positif dan negatif.

In [ ]:
def show_info_label(x):
  return Counter(x)

In [ ]:
# oversampling
oversample = SMOTE()
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)
# info label
show_info_label(y_train_over)

Counter({-1: 500, 0: 500, 1: 500})

In [ ]:
# undersampling
undersample = RandomUnderSampler()
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)
# info label
show_info_label(y_train_under)

Counter({-1: 144, 0: 144, 1: 144})

In [ ]:
data = {
    'original': (X_train, y_train),
    'oversampling': (X_train_over, y_train_over),
    'undersampling': (X_train_under, y_train_under)
    }

In [ ]:
print("original data : ", len(data['original'][0]))
print("oversampling data : ", len(data['oversampling'][0]))
print("undersampling data : ", len(data['undersampling'][0]))

original data :  800
oversampling data :  1470
undersampling data :  444


## 3.7 Implementasi SVM dan Evaluasi

### 3.7.1 Implementasi SVM dengan Kernel RBF

In [ ]:
# nilai pada setiap parameter pada kernel rbf yang akan digunakan
param = {
    'kernel': ['rbf'],
    'C': [0.5, 1, 10, 100],
    'gamma': ['scale', 1, 0.1, 0.01, 0.001]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

# mencari parameter terbaik dengan grisearch
grid_search = GridSearchCV(estimator=SVC(random_state=0),
                           param_grid=param,
                           n_jobs=8,
                           verbose=1,
                           scoring='accuracy')

### 3.7.2 Evaluasi pada Data Asli

In [ ]:
# menggunakan data asli
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=SVC(random_state=0), n_jobs=8,
             param_grid={'C': [0.5, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=1)

In [ ]:
# akurasi tertinggi yang didapat
grid_search.best_score_

0.6425000000000001

In [ ]:
# parameter terbaik pada data asli
grid_search.best_estimator_.get_params()

{'C': 10,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 0.1,
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 0,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
# prediksi pada data test
y_pred = grid_search.predict(X_test)
# tampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.74      0.86      0.80       132
           0       0.42      0.43      0.43        30
           1       0.67      0.26      0.38        38

    accuracy                           0.69       200
   macro avg       0.61      0.52      0.53       200
weighted avg       0.68      0.69      0.66       200



### 3.7.3 Evaluasi pada Data Undersampling

In [ ]:
# menggunakan data undersampling
grid_search.fit(X_train_under, y_train_under)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=SVC(random_state=0), n_jobs=8,
             param_grid={'C': [0.5, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=1)

In [ ]:
# menampilkan parameter terbaik
grid_search.best_estimator_.get_params()

{'C': 1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 0,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
# prediksi pada data test
y_pred = grid_search.predict(X_test)
# menampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.84      0.53      0.65       132
           0       0.39      0.80      0.52        30
           1       0.42      0.61      0.49        38

    accuracy                           0.58       200
   macro avg       0.55      0.65      0.56       200
weighted avg       0.69      0.58      0.60       200



### 3.7.4 Evaluasi pada Data Oversampling

In [ ]:
# menggunakan data hasil oversampling
grid_search.fit(X_train_over, y_train_over)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=SVC(random_state=0), n_jobs=8,
             param_grid={'C': [0.5, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=1)

In [ ]:
# parameter terbaik untuk data
grid_search.best_estimator_.get_params()

{'C': 10,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 0,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
# skor akurasi tertinggi
grid_search.best_score_

0.8945578231292517

In [ ]:
# prediksi pada data test
y_pred = grid_search.predict(X_test)
# tampilkan hasil
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.95      0.83       132
           0       0.63      0.40      0.49        30
           1       0.60      0.16      0.25        38

    accuracy                           0.71       200
   macro avg       0.65      0.50      0.52       200
weighted avg       0.69      0.71      0.67       200

